## Класифікатори кухонь 2

У цьому другому уроці класифікації ми дослідимо `більше способів` класифікувати категоричні дані. Ми також розглянемо наслідки вибору одного класифікатора над іншим.

### [**Тест перед лекцією**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Передумови**

Ми припускаємо, що ви завершили попередні уроки, оскільки будемо використовувати деякі концепції, які ми вивчали раніше.

Для цього уроку нам знадобляться наступні пакети:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) — це [колекція пакетів для R](https://www.tidyverse.org/packages), створена для того, щоб зробити роботу з даними швидшою, простішою та цікавішою!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) — це [фреймворк](https://www.tidymodels.org/packages/) для моделювання та машинного навчання, який складається з колекції пакетів.

-   `themis`: [пакет themis](https://themis.tidymodels.org/) надає додаткові кроки для роботи з незбалансованими даними.

Ви можете встановити їх за допомогою:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Або ж скрипт нижче перевірить, чи є у вас необхідні пакети для завершення цього модуля, і встановить їх, якщо вони відсутні.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Карта класифікації**

У нашому [попередньому уроці](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1) ми намагалися відповісти на запитання: як обрати між кількома моделями? Значною мірою це залежить від характеристик даних і типу задачі, яку ми хочемо вирішити (наприклад, класифікація чи регресія?).

Раніше ми дізналися про різні варіанти класифікації даних за допомогою шпаргалки від Microsoft. Фреймворк машинного навчання Python, Scikit-learn, пропонує подібну, але більш деталізовану шпаргалку, яка може ще більше допомогти звузити вибір оцінювачів (інша назва для класифікаторів):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Порада: [відвідайте цю карту онлайн](https://scikit-learn.org/stable/tutorial/machine_learning_map/) і натискайте на шляхи, щоб прочитати документацію.
>
> [Сайт довідки Tidymodels](https://www.tidymodels.org/find/parsnip/#models) також надає чудову документацію про різні типи моделей.

### **План** 🗺️

Ця карта дуже корисна, якщо ви добре розумієте свої дані, адже ви можете «пройтися» її шляхами до рішення:

-   У нас є \>50 зразків

-   Ми хочемо передбачити категорію

-   У нас є мічені дані

-   У нас менше ніж 100K зразків

-   ✨ Ми можемо обрати Linear SVC

-   Якщо це не спрацює, оскільки у нас числові дані

    -   Ми можемо спробувати ✨ KNeighbors Classifier

        -   Якщо це не спрацює, спробуйте ✨ SVC та ✨ Ensemble Classifiers

Це дуже корисний шлях для слідування. Тепер давайте перейдемо до справи, використовуючи [tidymodels](https://www.tidymodels.org/) — фреймворк для моделювання: узгоджена та гнучка колекція пакетів R, розроблена для заохочення хороших статистичних практик 😊.

## 2. Розділіть дані та вирішіть проблему незбалансованого набору даних.

З наших попередніх уроків ми дізналися, що існує набір спільних інгредієнтів у наших кухнях. Також було помітно нерівномірний розподіл кількості кухонь.

Ми вирішимо ці проблеми, виконавши наступне:

-   Видалимо найпоширеніші інгредієнти, які створюють плутанину між різними кухнями, використовуючи `dplyr::select()`.

-   Використаємо `recipe`, який попередньо обробляє дані, щоб підготувати їх до моделювання, застосовуючи алгоритм `over-sampling`.

Ми вже розглядали це в попередньому уроці, тому це має бути легко 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Робота з незбалансованими даними

Незбалансовані дані часто негативно впливають на продуктивність моделі. Багато моделей працюють найкраще, коли кількість спостережень є рівною, і тому їм важко справлятися з незбалансованими даними.

Існує два основних способи вирішення проблеми незбалансованих наборів даних:

-   додавання спостережень до меншої класу: `Over-sampling`, наприклад, використання алгоритму SMOTE, який синтетично генерує нові приклади меншої класу, використовуючи найближчих сусідів цих випадків.

-   видалення спостережень із більшої класу: `Under-sampling`

У нашому попередньому уроці ми продемонстрували, як працювати з незбалансованими наборами даних, використовуючи `recipe`. Recipe можна уявити як план, який описує, які кроки слід застосувати до набору даних, щоб підготувати його до аналізу. У нашому випадку ми прагнемо до рівномірного розподілу кількості наших кухонь у `training set`. Давайте перейдемо до справи.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Тепер ми готові тренувати моделі 👩‍💻👨‍💻!

## 3. Поза межами моделей багатономінальної регресії

У нашому попередньому уроці ми розглянули моделі багатономінальної регресії. Давайте дослідимо більш гнучкі моделі для класифікації.

### Машини опорних векторів

У контексті класифікації `Машини опорних векторів` — це метод машинного навчання, який намагається знайти *гіперплощину*, що "найкраще" розділяє класи. Розглянемо простий приклад:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ не розділяє класи. H2~ розділяє, але лише з невеликим відступом. H3~ розділяє їх з максимальним відступом.

#### Лінійний класифікатор на основі опорних векторів

Кластеризація на основі опорних векторів (SVC) є частиною сімейства методів машинного навчання, які використовують опорні вектори. У SVC гіперплощина обирається таким чином, щоб правильно розділити `більшість` навчальних спостережень, але `може неправильно класифікувати` деякі з них. Дозволяючи деяким точкам бути на неправильній стороні, SVM стає більш стійким до викидів, що забезпечує кращу узагальненість для нових даних. Параметр, який регулює це порушення, називається `cost` і має значення за замовчуванням 1 (див. `help("svm_poly")`).

Давайте створимо лінійний SVC, встановивши `degree = 1` у поліноміальній моделі SVM.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Тепер, коли ми зафіксували етапи попередньої обробки та специфікацію моделі у *workflow*, можемо перейти до навчання лінійного SVC і одночасно оцінити результати. Для оцінки продуктивності давайте створимо набір метрик, який буде враховувати: `accuracy`, `sensitivity`, `Positive Predicted Value` та `F Measure`.

> `augment()` додасть стовпець(и) з прогнозами до наданих даних.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Машина опорних векторів

Машина опорних векторів (SVM) є розширенням класифікатора опорних векторів, яке дозволяє враховувати нелінійну межу між класами. По суті, SVM використовують *трюк з ядром*, щоб розширити простір ознак і адаптуватися до нелінійних взаємозв'язків між класами. Однією з популярних і надзвичайно гнучких функцій ядра, яку використовують SVM, є *радіальна базисна функція*. Давайте подивимося, як вона працюватиме на наших даних.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Набагато краще 🤩!

> ✅ Будь ласка, перегляньте:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Практичне машинне навчання з R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), Вступ до статистичного навчання з застосуванням R
>
> для додаткового читання.

### Класифікатори найближчих сусідів

*K*-найближчий сусід (KNN) — це алгоритм, у якому кожне спостереження прогнозується на основі його *схожості* з іншими спостереженнями.

Давайте застосуємо його до наших даних.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Здається, що ця модель працює не дуже добре. Можливо, зміна аргументів моделі (див. `help("nearest_neighbor")`) покращить її продуктивність. Обов’язково спробуйте це зробити.

> ✅ Будь ласка, ознайомтеся з:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> щоб дізнатися більше про класифікатори *K*-найближчих сусідів.

### Ансамблеві класифікатори

Ансамблеві алгоритми працюють, комбінуючи кілька базових оцінювачів для створення оптимальної моделі шляхом:

`bagging`: застосування *усереднювальної функції* до набору базових моделей

`boosting`: побудови послідовності моделей, які покращують одна одну для підвищення точності прогнозування.

Давайте почнемо з моделі Random Forest, яка створює велику кількість дерев рішень, а потім застосовує усереднювальну функцію для отримання кращої загальної моделі.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Молодець 👏!

Давайте також спробуємо модель Boosted Tree.

Boosted Tree визначає ансамблевий метод, який створює серію послідовних дерев рішень, де кожне дерево залежить від результатів попередніх дерев, намагаючись поступово зменшити помилку. Він зосереджується на вагах неправильно класифікованих елементів і коригує підгонку для наступного класифікатора, щоб виправити помилки.

Існують різні способи підгонки цієї моделі (див. `help("boost_tree")`). У цьому прикладі ми підженемо Boosted trees за допомогою двигуна `xgboost`.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Будь ласка, ознайомтеся:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Досліджує модель AdaBoost, яка є гарною альтернативою xgboost.
>
> щоб дізнатися більше про ансамблеві класифікатори.

## 4. Додатково - порівняння кількох моделей

Ми побудували досить багато моделей у цій лабораторній роботі 🙌. Може стати виснажливим або обтяжливим створювати багато робочих процесів із різних наборів препроцесорів та/або специфікацій моделей, а потім по одній обчислювати метрики продуктивності.

Давайте подивимося, чи зможемо ми вирішити це, створивши функцію, яка навчає список робочих процесів на навчальному наборі, а потім повертає метрики продуктивності на основі тестового набору. Ми скористаємося `map()` та `map_dfr()` із пакету [purrr](https://purrr.tidyverse.org/), щоб застосувати функції до кожного елемента списку.

> [`map()`](https://purrr.tidyverse.org/reference/map.html) дозволяє замінити багато циклів for кодом, який є більш лаконічним і легшим для читання. Найкраще місце для вивчення функцій [`map()`](https://purrr.tidyverse.org/reference/map.html) — це [розділ про ітерацію](http://r4ds.had.co.nz/iteration.html) у книзі "R for Data Science".


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) пакет дозволяє користувачам створювати та легко налаштовувати велику кількість моделей, але в основному призначений для роботи з методами повторного вибіркового моделювання, такими як `перехресна перевірка`, підхід, який ми ще не розглядали.

## **🚀Виклик**

Кожна з цих технік має велику кількість параметрів, які ви можете налаштовувати, наприклад, `cost` у SVM, `neighbors` у KNN, `mtry` (випадково вибрані предиктори) у Random Forest.

Дослідіть стандартні параметри кожної з них і подумайте, що зміна цих параметрів може означати для якості моделі.

Щоб дізнатися більше про конкретну модель та її параметри, скористайтеся: `help("model")`, наприклад, `help("rand_forest")`.

> На практиці ми зазвичай *оцінюємо* *найкращі значення* цих параметрів, тренуючи багато моделей на `змодельованому наборі даних` і вимірюючи, наскільки добре ці моделі працюють. Цей процес називається **налаштуванням**.

### [**Тест після лекції**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Огляд і самостійне навчання**

У цих уроках багато спеціальних термінів, тому приділіть хвилину, щоб переглянути [цей список](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) корисної термінології!

#### ДЯКУЄМО:

[`Елісон Хорст`](https://twitter.com/allison_horst/) за створення дивовижних ілюстрацій, які роблять R більш доступним і цікавим. Знайдіть більше ілюстрацій у її [галереї](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Кессі Брев'ю](https://www.twitter.com/cassieview) та [Джен Лупер](https://www.twitter.com/jenlooper) за створення оригінальної версії цього модуля на Python ♥️

Щасливого навчання,

[Ерік](https://twitter.com/ericntay), Золотий студентський амбасадор Microsoft Learn.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Ілюстрація від @allison_horst</figcaption>



---

**Відмова від відповідальності**:  
Цей документ було перекладено за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, зверніть увагу, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ мовою оригіналу слід вважати авторитетним джерелом. Для критично важливої інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникли внаслідок використання цього перекладу.
